# Metadata Filtering
- 벡터 유사도 계산 전에(또는 함께) 문서의 메타데이터 조건으로 검색 대상을 제한하는 기법

In [ ]:
%pip install -Uq python-dotenv langchain langchain-openai langchain-pinecone langchain-classic langchain-community lark

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")
os.environ['PINECONE_API_KEY'] = os.getenv("pinecone_key")
os.environ['COHERE_API_KEY'] = os.getenv('cohere_key')

## 데이터 준비

In [ ]:
import pandas as pd

documents_df = pd.read_csv('documents_meta.csv')
documents_df

In [ ]:
import pandas as pd

queries_df = pd.read_csv('queries_meta_v2.csv')
queries_df

## 벡터 인덱스 생성

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone() # pinecone api key 인증
print(pc.list_indexes().names()) # ['winemag-data-130k-v2', 'pinecone-first']


# 인덱스명 ir
if 'ir-meta' not in pc.list_indexes().names():
    pc.create_index(
        name='ir-meta',
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            region='us-east-1',
            cloud='aws'
        )
    )
    print('ir-meta인덱스가 생성되었습니다.')
else:
    print('ir-meta인덱스가 이미 존재합니다.')



## 벡터 서치

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding_model = OpenAIEmbeddings()
vector_store = PineconeVectorStore(
    index_name='ir-meta',
    embedding=embedding_model
 )

## Document Upsert
- 임베딩변환할 텍스트외의 컬럼은 metadata 속성(dict)에 저장되어 있어야 한다.

In [ ]:
from langchain_core.documents import Document

docs_to_upsert = []

for idx, row in documents_df.iterrows():
    categories = row['category'].split(';')
    metadata = {
        'doc_id': row['doc_id'],
        'author': row['author'],
        'category': categories
    }
    doc = Document(
        page_content=row['content'],
        metadata=metadata
    )
    docs_to_upsert.append(doc)  # Document 리스트

print(len(docs_to_upsert))

In [ ]:
# Document 리스트를 Pinecone(ir-meta)에 업서트
vector_store.add_documents(docs_to_upsert)

## 메타데이터 필터를 사용한 벡터 검색

https://python.langchain.com/docs/integrations/vectorstores/

LangChain의 `vectorstore.similarity_search`에서 **filter**를 사용하는 방법은 벡터스토어 종류에 따라 약간 다르지만, 기본적으로 **문서의 메타데이터(metadata)를 조건으로 검색 결과를 제한**하는 데 사용한다.

**고급 필터링 (MongoDB 스타일 연산자)**

몇몇 벡터스토어(예: FAISS, Chroma 등)는 **MongoDB 스타일의 연산자**를 지원한다. 예를 들어, 특정 값과 일치하는 것뿐만 아니라, 크거나, 작거나, 리스트 내 포함 등 다양한 조건을 줄 수 있다.

지원 연산자 예시:
- `$eq`: 같음
- `$neq`: 같지 않음
- `$gt`: 큼
- `$lt`: 작음
- `$gte`: 크거나 같음
- `$lte`: 작거나 같음
- `$in`: 리스트 내 포함
- `$nin`: 리스트 내 미포함
- `$and`, `$or`, `$not`: 논리 연산

In [ ]:
query = ''
results = vector_store.similarity_search(
    query,
    k=5,
    filter={
        'category': {
            '$in': ['역사', '여행']  # categori가 역사 또는 여행인 문서만 검색
        }
    }
)

for doc in results:
    print(doc.metadata['doc_id'])
    print(doc.metadata['category'])
    print(doc.page_content)
    print()


In [ ]:
query = ''
results = vector_store.similarity_search(
    query,
    k=5,
    filter={
        'author': {
            '$eq': '이영희'  # author가 '이영희'인 문서만 검색
        }
    }
)

for doc in results:
    print(doc.metadata['doc_id'])
    print(doc.metadata['author'])
    print(doc.page_content)
    print()


## SelfQueryRetriever

**SelfQueryRetriever**는 사용자의 자연어 질문을 분석하여 쿼리 자체에 포함된 **메타데이터 필터**와 **벡터 검색용 쿼리**를 분리하여 실행하는 검색 방식이다.

일반적인 벡터 검색은 질문의 '의미'만으로 데이터를 찾지만, SelfQueryRetriever는 LLM을 활용해 질문에서 조건(필터)을 추출한다.

**1. 작동 원리**

사용자가 "2023년 이후에 개봉한 평점 8.5 이상의 SF 영화 찾아줘"라고 질문하면, SelfQueryRetriever는 내부적으로 다음과 같은 과정을 거친다.

1. **질문 분석**: LLM이 질문을 해석하여 검색어(Query)와 필터 조건(Metadata Filter)을 생성한다.
* **검색어**: "SF 영화"
* **필터**: `release_year > 2023 AND rating >= 8.5`


2. **쿼리 실행**: 생성된 필터를 벡터 데이터베이스(Vector DB)의 메타데이터에 적용함과 동시에 검색어로 유사도 검색을 수행한다.
3. **결과 반환**: 두 조건이 모두 충족된 결과물만 사용자에게 전달한다.

**2. 주요 구성 요소**

| 구성 요소 | 설명 |
| --- | --- |
| **Document Content Description** | 데이터셋이 어떤 내용을 담고 있는지 LLM에게 설명하는 텍스트이다. |
| **Metadata Field Info** | 각 메타데이터 필드의 이름, 타입(정수, 문자열 등), 의미를 정의한 정보이다. |
| **LLM** | 자연어 질문을 구조화된 쿼리(필터)로 변환하는 역할을 한다. |
| **Vector Store** | 실제 데이터와 메타데이터가 저장된 저장소이다. |

**3. 왜 사용하는가?**

* **정확도 향상**: "최근에 나온"이나 "특정 점수 이상" 같은 정량적 조건은 의미론적 유사성만으로는 정확히 필터링하기 어렵다. 이를 논리적 필터로 처리함으로써 검색 결과의 신뢰도를 높인다.
* **자연스러운 인터페이스**: 사용자가 SQL이나 복잡한 필터 문법을 배울 필요 없이, 일상적인 언어로 정교한 검색이 가능하다.
* **효율성**: 불필요한 데이터를 벡터 검색 단계에서 미리 배제하여 검색 범위를 좁힐 수 있다.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_classic.retrievers import SelfQueryRetriever
from langchain_classic.chains.query_constructor.schema import AttributeInfo

llm = init_chat_model('gpt-4.1-mini', temperature=0)

# 메타데이터 필드 정보
metadata_field_info = [
    AttributeInfo(name='doc_id', type='string', description='문서 식별 번호'),
    AttributeInfo(name='author', type='string', description='문서를 작성한 저자의 이름'),
    AttributeInfo(name='category', type='list[string]', description='문서의 분류 목록'),
    AttributeInfo(name='text', type='string', description='원문 텍스트'),
]

# 문서 내용 정보
document_content_description = '다양한 분류의 텍스트 데이터셋'

self_query_retriever = SelfQueryRetriever.from_llm(  # 자연어 쿼리를 필터+검색으로 변환
    llm=llm,
    vectorstore=vector_store,
    document_contents=document_content_description,  # 문서 내용 설명
    metadata_field_info=metadata_field_info          # 사용 가능한 메타데이터 스키마
)

## 메타데이터 필터링 수행

In [ ]:
from tqdm.auto import tqdm

results_list = []
self_query_results = {} # 평가용 (qid: doc_id 리스트)

for idx, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
    qid = row['query_id']
    query_text = row['query_text']
    docs = self_query_retriever.invoke(query_text)  # Self-Query 기반 검색 실행
    structured_query = self_query_retriever.query_constructor.invoke({'query': query_text})  # LLM이 생성한 구조화 쿼리
    self_query_results[qid] = [doc.metadata['doc_id'] for doc in docs[:5]]  # 상위 5개 doc_id 저장

    # DataFrame시각화
    results_list.append({
        'query_id': qid,
        'query_text': query_text,  # 원본 쿼리
        'structured_query': str(structured_query),    # 생성된 구조화 쿼리 문자열
        'retrieved_doc_ids': self_query_results[qid]  # 검색된 문서 ID 목록
    })

In [ ]:
pd.set_option('display.max_colwidth', None)  # 긴 텍스트 컬럼 전체 표시 설정
results_df = pd.DataFrame(results_list)      # 수집한 결과 리스트를 DataFrame으로 변환
results_df

## 검색성능 평가

**평가지표 설명**

* **Precision\@k**: 상위 k개의 검색 결과 중에 진짜 필요한 문서가 얼마나 있는지를 측정한다.
  - 예컨대 k=5일 때, 결과 5개 중 관련 문서가 2개면 Precision\@5 = 2/5 = 0.4다.

* **Recall\@k**: 전체 관련 문서 중에서 상위 k개 안에 얼마나 많이 들어왔는지를 본다.
  - 예컨대 전체 관련 문서가 4개이고, 그중 3개가 상위 5개 안에 들어오면 Recall\@5 = 3/4 = 0.75다.


* **MRR (Mean Reciprocal Rank)**:
  사용자가 제시한 여러 질의에서, 각 질의별로 “첫 번째 관련 문서”가 나온 순위의 역수를 구한 뒤 평균낸 것이다.
  **예시**

  * 질의 A: 첫 관련 문서가 2위 → RR = 1/2 = 0.5
  * 질의 B: 첫 관련 문서가 3위 → RR = 1/3 ≈ 0.333
  * 질의 C: 첫 관련 문서가 1위 → RR = 1/1 = 1
  * 이 세 질의의 MRR = (0.5 + 0.333 + 1) / 3 ≈ 0.611

* **AP (Average Precision)**:
  한 질의 결과 리스트를 순서대로 훑어가며, 관련 문서를 만날 때마다 그 시점까지의 Precision을 계산한 뒤, 관련 문서 개수로 나눈 값이다.
  **예시** (관련 문서 3개가 있고, 순위 2, 4, 5위에 위치한 경우)

  1. 2위에서 첫 관련 문서 발견 → Precision\@2 = 1/2 = 0.50
  2. 4위에서 두 번째 관련 문서 발견 → Precision\@4 = 2/4 = 0.50
  3. 5위에서 세 번째 관련 문서 발견 → Precision\@5 = 3/5 = 0.60
     AP = (0.50 + 0.50 + 0.60) / 3 ≈ 0.533

In [ ]:
import numpy as np

# 참조문서 답안 파싱
def parse_relevant(relevant_str) -> dict[str, int]:
    """
    relevant_str = 'D1=3;D4=1;D30=1' -> {'D1': 3, 'D4': 1, 'D30': 1}
    """
    pairs = relevant_str.split(';')
    rel_dict = {}
    for pair in pairs:
        doc_id, grade = pair.split('=')
        rel_dict[doc_id] = int(grade)
    return rel_dict


# 평가지표 계산
def compute_metrics(predicted, relevant_dict, k=5) -> tuple[float, float, float, float]:
    # Precision@k
    hits = sum([1 for doc in predicted[:k] if doc in relevant_dict])
    precision = hits / k

    # Recall@k
    total_relevant = len(relevant_dict) # {'D1': 3, 'D12': 2, ...}
    recall = hits / total_relevant if total_relevant > 0 else 0

    # MRR 예측치중 첫 관련문서 순위점수
    rr = 0
    for idx, doc in enumerate(predicted):
        if doc in relevant_dict:
            rr = 1 / (idx + 1)
            break

    # AP(MAP를 위한 계산)
    num_correct = 0
    precisions = []
    for idx, doc in enumerate(predicted[:k]):
        if doc in relevant_dict:
            num_correct += 1
            precisions.append(num_correct / (idx + 1))
    ap = np.mean(precisions) if precisions else 0
    return precision, recall, rr, ap


# 평가함수
def evaluate_all(method_results, queries_df, k=5):
    prec_list, rec_list, rr_list, ap_list = [], [], [], []

    for idx, row in queries_df.iterrows():
        qid = row['query_id']
        relevant_dict = parse_relevant(row['relevant_doc_ids'])
        predicted = method_results[qid]
        p, r, rr, ap = compute_metrics(predicted, relevant_dict, k)
        prec_list.append(p)
        rec_list.append(r)
        rr_list.append(rr)
        ap_list.append(ap)

    return {
        'P@k': np.mean(prec_list),
        'R@k': np.mean(rec_list),
        'MRR': np.mean(rr_list),
        'MAP': np.mean(ap_list),
    }


## 검색 수행

In [ ]:
# Dense(VectorStore 검색) 결과 수집
from tqdm.auto import tqdm

dense_results = {}

for idx, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
    qid = row['query_id']
    query_text = row['query_text']
    docs = vector_store.similarity_search(query_text, k=5)
    dense_results[qid] = [doc.metadata['doc_id'] for doc in docs]

## 성능평가

In [ ]:
dense_metrics = evaluate_all(dense_results, queries_df, k=5)
self_query_metrics = evaluate_all(self_query_results, queries_df, k=5)

In [ ]:
import pandas as pd
metrics_df = pd.DataFrame({
    'Metrics': ['P@5', 'R@5', 'MRR', 'MAP'],
    'Dense': [dense_metrics['P@k'], dense_metrics['R@k'], dense_metrics['MRR'], dense_metrics['MAP']],
    'Metadata': [self_query_metrics['P@k'], self_query_metrics['R@k'], self_query_metrics['MRR'], self_query_metrics['MAP']],
})
metrics_df


In [ ]:
# 시각화
import matplotlib.pyplot as plt

metrics = ['P@5', 'R@5', 'MRR', 'MAP']
dense_vals = [dense_metrics['P@k'], dense_metrics['R@k'], dense_metrics['MRR'], dense_metrics['MAP']]
metadata_vals = [self_query_metrics['P@k'], self_query_metrics['R@k'], self_query_metrics['MRR'], self_query_metrics['MAP']]

x = range(len(metrics))

plt.figure(figsize=(8, 4))
plt.plot(x, dense_vals, marker='o', label='Dense')
plt.plot(x, metadata_vals, marker='s', label='Metadata')
plt.xticks(x, metrics)
plt.ylim(0, 1.1)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Dense vs Metadata')
plt.legend()
plt.grid()
plt.show()

## 왜 이런 결과가 나왔나?

- **Dense Retrieval**
  - 문서의 **의미적 유사도**만을 기준으로 판단
  - 작성자(author), 카테고리(category)와 같은 **구조적 조건을 직접 반영하지 못함**

- **Self-Query Retriever (Metadata 기반)**
  - 자연어 질의에서 **조건을 자동으로 추출**
  - 추출된 조건을 **메타데이터 필터로 강제 적용**하여 검색

예를 들어,

> **“이영희가 쓴 여행 관련 글”**

과 같은 질의는  
- Dense Retrieval에게는 의미적으로 **모호한 질문**이지만  
- Metadata 검색에게는 **명확한 조건 기반 질의**가 된다.

---

## 실무 결론 (중요)

### 권장 검색 파이프라인
Self-Query / Metadata Filtering  
↓  
Dense Retrieval  
↓  
ReRank (선택)

- **조건이 명확한 질의** → Metadata 검색 우선
- **의미적 탐색 중심 질의** → Dense Retrieval 활용
- **실제 서비스 환경** → 두 방식을 결합한 Hybrid 전략 적용

---

## 한 줄 결론 (교안 / 발표용)

> **구조적 조건이 포함된 질의에서는 Dense Retrieval보다 Metadata 기반 검색이 압도적으로 높은 정확도를 보인다.**